In [ ]:
!pip install virtualenv
#INSTALLING VENV

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 28.5 MB/s eta 0:00:00


In [ ]:
!virtualenv myenv
#INITIALISING VENV

created virtual environment CPython3.10.12.final.0-64 in 1071ms
  creator CPython3Posix(dest=/content/myenv, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==24.1, setuptools==70.1.0, wheel==0.43.0
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
!source /content/myenv/bin/activate

In [ ]:
!pip install gym==0.21.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 28.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


In [ ]:
!pip install ta
!pip install yfinance

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=782ce00466bc4a9b791a804622b34d10cb225c02c881f7b611dda11813a3b32f
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
!pip install pyfolio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.8/52.8 kB 5.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
Using cached jedi-0.19.1-py2.py3-none-any.whl (1.6 MB)
  Created wheel for pyfolio: filename=pyfolio-0.9.2-py3-none-any.whl size=88657 sha256=fe85102f0118f816d359e085990ab5a258cc2650d53f91b2b37bb7cd2d83ad59
  Stored in directory: /root/.cache/pip/wheels/71/38/bc/e53700cfd8b0ad6b539d2fbaaf060ed8a299e7622a5b86ef42
  Created wheel for empyrical: filename=empyrical-0.5.5-py3-none-any.whl size=39750 sha256=ccdc78b7e819c9fbba83a0ce8c1fd436e4bb2ccacb3c1b923d3fa76f9644539f
  Stored in directory: /root/.cache/pip/wheels/0e/2e/f2/d6d2d9a1eb8fbbd9949bb5d4c00f753e3b74e5bd7ed10b1d36
Successfully built pyfolio empyrical


In [ ]:
import gym
import numpy as np
import pandas as pd
import yfinance as yf
import ta

import pyfolio as pf
import torch as th
import torch.nn as nn


/usr/local/lib/python3.10/dist-packages/pandas_datareader/compat/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  PANDAS_VERSION = LooseVersion(pd.__version__)
/usr/local/lib/python3.10/dist-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [ ]:
class StockTradingEnv(gym.Env):
    def __init__(self, df):
        super(StockTradingEnv, self).__init__()
        self.df = df #for accessing stock data
        self.action_space = gym.spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32) #action space is one dim array
        self.observation_space = gym.spaces.Box(low=-np.inf, high=np.inf, shape=(5,), dtype=np.float32) #obs space is five dim array
        self.reset()

    def reset(self):    #resets itself
        self.current_step = 0
        self.cash = 10000  # Initial cash
        self.shares_held = 0
        return self._next_observation()

    def _next_observation(self): #obs array contains these
        obs = np.array([
            self.df.iloc[self.current_step]['Close'],
            self.cash,
            self.shares_held,
            self.df.iloc[self.current_step]['MACD'], #moving average convergence divergence
            self.df.iloc[self.current_step]['RSI']  #relative strength index
        ])
        return obs

    def step(self, action):
        current_price = self.df.iloc[self.current_step]['Close']
        action = action[0]

        # # Buy shares
        # if action > 0:
        #     shares_bought = self.cash // current_price
        #     self.cash -= shares_bought * current_price
        #     self.shares_held += shares_bought

        # # Sell shares
        # elif action < 0:
        #     self.cash += self.shares_held * current_price
        #     self.shares_held = 0

        if action > 0:  # buy
              n_shares = self.cash * action /  current_price
              self.cash -= n_shares * current_price
              self.shares_held += n_shares
        elif action < 0:  # sell
              n_shares = -self.shares_held * action
              self.cash += n_shares * current_price
              self.shares_held-= n_shares


        self.current_step += 1
        done = self.current_step >= len(self.df) - 1
        reward = self.cash + self.shares_held * current_price

        obs = self._next_observation()
        return obs, reward, done, {}

    def render(self, mode='human', close=False):
        pass


In [ ]:
# Download data
nifty_ticker = '^NSEI'

# Download Nifty 50 data
df = yf.download(nifty_ticker, start='2023-08-10', end='2024-08-10')

# Calculate MACD and RSI
df['MACD'] = ta.trend.macd(df['Close'])
df['RSI'] = ta.momentum.rsi(df['Close'])
df = df.dropna()


[*********************100%%**********************]  1 of 1 completed


In [ ]:
!pip install stable-baselines3[extra]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 16.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached Farama_Notifications-0.0.4-py3-none-any.whl.metadata (558 bytes)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
train_df = df[:int(0.6*len(df))]
validate_df = df[int(0.6*len(df)):int(0.8*len(df))]
test_df = df[int(0.8*len(df)):]

train_env = StockTradingEnv(train_df)
validate_env = StockTradingEnv(validate_df)
test_env = StockTradingEnv(test_df)

n_actions = train_env.action_space.shape[-1]
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MlpPolicy", train_env, action_noise=action_noise, verbose=1)
model.learn(total_timesteps=50000)


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/vec_env/patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 130      |
|    ep_rew_mean     | 1.34e+06 |
| time/              |          |
|    episodes        | 4        |
|    fps             | 176      |
|    time_elapsed    | 2        |
|    total_timesteps | 520      |
| train/             |          |
|    actor_loss      | -3.2e+04 |
|    critic_loss     | 1.56e+07 |
|    learning_rate   | 0.001    |
|    n_updates       | 419      |
---------------------------------
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 130       |
|    ep_rew_mean     | 1.34e+06  |
| time/              |           |
|    episodes        | 8         |
|    fps             | 144       |
|    time_elapsed    | 7         |
|    total_timesteps | 1040      |
| train/             |           |
|    actor_loss      | -5.68e+04 |
|    critic_loss     | 1.94e+07  |
|    learning_rate   | 0.001     |
|    n_updates       | 939       |


In [ ]:
mean_reward, std_reward = evaluate_policy(model, validate_env, n_eval_episodes=10)
print(f"Validation Mean reward: {mean_reward} +/- {std_reward}")

obs = test_env.reset()
done = False
while not done:
    action, _states = model.predict(obs)
    obs, reward, done, info = test_env.step(action)

portfolio_value = test_env.cash + test_env.shares_held * test_df.iloc[test_env.current_step]['Close']
print(f"Final Portfolio Value: {portfolio_value}")

# Calculate Sharpe Ratio
returns = test_env.df['Close'].pct_change().dropna()
# pf.create_returns_tear_sheet(returns)


/usr/local/lib/python3.10/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Validation Mean reward: 430381.7060546875 +/- 0.0
Final Portfolio Value: 10462.57735193014


In [ ]:
print(test_env.shares_held )

0.4293660552756803


In [ ]:
!pip install empyrical


ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 3070, in _dep_map
    return self.__dep_map
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/pkg_resources/__init__.py", line 2863, in __getattr__
    raise AttributeError(attr)
AttributeError: _DistInfoDistribution__dep_map

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
  File "/usr/local/lib/python3.10/dist-packages/pip/_in

In [ ]:
import pandas as pd
import empyrical as ep


In [ ]:
sharpe_ratio = ep.sharpe_ratio(returns)


In [ ]:
returns

,Close
Date,
2024-06-10,-0.001329
2024-06-11,0.000243
2024-06-12,0.002497
2024-06-13,0.003256
2024-06-14,0.002851
2024-06-18,0.003933
2024-06-19,-0.001779
2024-06-20,0.002169
2024-06-21,-0.002796


In [ ]:
print(f"Sharpe Ratio: {sharpe_ratio}")

Sharpe Ratio: 2.4403743544104617
